Using the Array prototype's `map()` method instead of a `for` loop to transform an array:

In [1]:
var myArray = [0, 1, 2, 3, 4];
var myArrayTransformed = myArray.map(function(arrayElement) {
        return arrayElement * 2;
    });
console.log(myArrayTransformed); // => [ 0, 2, 4, 6, 8 ]

[ 0, 2, 4, 6, 8 ]


undefined

In [6]:
function capStrElements(arr) {
    return arr.map(function(currVal) {
        return currVal.charAt(0).toUpperCase() +
               currVal.substring(1).toLowerCase();
    });
}

console.log(capStrElements(['foo', 'bar', 'baz'])); // => ['Foo', 'Bar', 'Baz']
console.log(capStrElements(['FOO', 'BAR', 'BAZ'])); // => ['Foo', 'Bar', 'Baz']

[ 'Foo', 'Bar', 'Baz' ]
[ 'Foo', 'Bar', 'Baz' ]


undefined

Using the Array prototype's `filter()` method to filter an array:

In [7]:
function filterEven(arr) {
    return arr.filter(function(currVal) {
        return currVal % 2 === 0;
    });
}

console.log(filterEven([0, 1, 2, 3, 4, 5])); // => [0, 2, 4]

[ 0, 2, 4 ]


undefined